<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/95_wind_speed_and_direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [1]:
%%capture
!pip install cartopy cfgrib

In [2]:
import cartopy.crs as ccrs
import imageio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from tqdm import trange

In [3]:
ds = xr.load_dataset(
    'March2024_windcomponents_NB.grib',
    engine='cfgrib',
    decode_timedelta=False,
)

In [4]:
lats = ds.variables['latitude']
lons = ds.variables['longitude']
lon, lat = np.meshgrid(lons, lats)

u10 = ds.sel(time='2024-03-10').u10.to_numpy()
v10 = ds.sel(time='2024-03-10').v10.to_numpy()
ws = np.sqrt(u10 ** 2 + v10 ** 2)

In [6]:
for i in trange(24):

    fig = plt.figure(figsize=(20, 15))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(resolution='10m', linewidth=1)

    c_inv = np.arange(0,14,1)
    plt.contourf(lon, lat, ws[i], c_inv, transform=ccrs.PlateCarree(), cmap=plt.cm.jet)
    plt.title(f'ERA5 Wind Speed and Direction for NB: 2024-03-10 {i:02}:00 hrs', size=16)
    cb = plt.colorbar(ax=ax, orientation='vertical', pad=0.02, aspect=16, shrink=0.8)
    cb.set_label('m/s',size=14,rotation=0,labelpad=15)
    cb.ax.tick_params(labelsize=10)
    qv = plt.quiver(lon, lat, u10[i], v10[i], scale=350, color='black')

    plt.savefig(f'{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

100%|██████████| 24/24 [00:47<00:00,  1.99s/it]


In [7]:
images = []
for filename in [f'{i}.png' for i in trange(24)]:
    img_data = imageio.v2.imread(filename)
    images.append(img_data)
imageio.mimsave('movie.gif', images, duration=200, loop=0)

100%|██████████| 24/24 [00:00<00:00, 77612.41it/s]
